In [1]:
#CET313 Codie Heaney - bh97rt
#Main Program

from neuralnet import NN
from Matrix import Matrix
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import numpy as np
import timeit
from tqdm import tqdm

#download and save the training data and test data sets to '../data'
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

#batch size for the dataset
total_batch = 64

#list of the categories each image falls into
labels_map = ["T-Shirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

#list of the target values for each category in identical order to the label list
targets = [[1,0,0,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0,0,0], [0,0,1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0,0,0], [0,0,0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0,0,0], [0,0,0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1,0,0], [0,0,0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,0,0,1]]

#create the neural network from the neural network library
nn = NN(784, 100, 10, 0.05)

#returns an array of the image data
def preproccess(image):
    
    train_image = image.squeeze() #removes unneccessary brackets from the list
    temp_matrix = Matrix.matrixToMatrixObj_return(train_image.numpy(), 28, 28)
    matrix_array = temp_matrix.toArray() #turns the image data into a usable list form for backpropogation
    return matrix_array

#function to begin the training process
def training(epochs, iterations):
    
    starttime = timeit.default_timer()
    
    #loops for how many times a new set of training should take place
    for ite in tqdm(range(iterations), desc = "Training...", ascii = False, ncols = 75):
        #loads the image data for training
        train_dataloader = DataLoader(training_data, batch_size=total_batch, shuffle=True) #selects a randomised batch of 64 from the dataset
        
        #create a list of image data with a corresponding list of the images labels
        train_features, train_labels = next(iter(train_dataloader))

        #loops for how many times training on this batch from the dataset should take place
        for e in range(epochs):
            for i in range(train_features.size()[0]):
                
                #prepare the image data and the target values
                matrix_array = preproccess(train_features[i])
                train_target = targets[train_labels[i]]
                
                #train the neural network
                nn.backpropagation(matrix_array, train_target)
                
    print("Training Time: ", timeit.default_timer() - starttime)
    
def testing():
    
    #load testing data
    test_dataloader = DataLoader(test_data, batch_size=total_batch, shuffle=True)
    
    #create a list of image data with a corresponding list of the images labels
    test_features, test_labels = next(iter(test_dataloader))
    
    total_error = 0
    
    for i in range(test_features.size()[0]):
        
        #preproccess image data and prepare labels for inaccuracy calculations
        matrix_array = preproccess(test_features[i])
        test_target = targets[test_labels[i]]
        
        #show neural network output and what it should've outputted
        prediction = nn.feedforward(matrix_array)
        print("\n\n\nPrediction: ", prediction[0])
        print("Actual: ", test_target)
        
        #calculate output error and show user
        target_matrix = Matrix.fromArray_return(test_target)
        error = Matrix.subtract_return(target_matrix, prediction[2])
        error = error.toArray()
        print("\nPrediction Inaccuracy: ", error)
        
        #show user a rounded to 2 decimal places error
        error_rounded = Matrix.subtract_return(target_matrix, prediction[2])
        error_rounded = error_rounded.toArray()
        round_array = np.array(error_rounded)
        round_array = np.round(round_array, decimals = 2)
        
        print("Prediction Inaccuracy(Rounded): ", round_array)
        
        highest_predict = 0
        for x in range(0, len(prediction[0])):
            if prediction[0][x] > prediction[0][highest_predict]:
                highest_predict = x
        
        if(labels_map[highest_predict] != labels_map[test_labels[i]]):
            total_error +=1
        
        #display the output and actual image label in text form
        print("\nNN Predicts: ", labels_map[highest_predict])
        print("Actual: ", labels_map[test_labels[i]])
    
    #calculate and show the total percentage it was wrong over all images fedforward
    error_perc = (total_error / test_features.size()[0]) * 100
    print("\n\nTotal Inaccuracy Percentage: " + str(error_perc) + "%")


#inputs for the user to change the number of hidden neurons/learning rate/epoch/iterations
def userInput():
    hiddens = input("Input number of hidden neurons:")
    lr = input("Set learning rate (0.0 to 1.0): ")
    nn = NN(784, int(hiddens), 10, float(lr))

    epoch = input("Set training Epoch: ")
    iteration = input("Set training iterations: ")
    training(int(epoch), int(iteration))

#uncomment below function for user input
#userInput()

#comment below function if using user input
training(3, 5)

#run test function
testing()

Training...: 100%|███████████████████████████| 5/5 [00:50<00:00, 10.14s/it]


Training Time:  50.70320139999967



Prediction:  [0.082650517686586, 0.190657347437325, 0.06731235892109173, 0.10529659912268344, 0.024223988761527496, 0.003006265841545637, 0.2145722159079363, 0.011688407531437682, 0.0024255011809505905, 0.007271788617458651]
Actual:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.082650517686586, -0.190657347437325, -0.06731235892109173, -0.10529659912268344, 0.9757760112384725, -0.003006265841545637, -0.2145722159079363, -0.011688407531437682, -0.0024255011809505905, -0.007271788617458651]
Prediction Inaccuracy(Rounded):  [-0.08 -0.19 -0.07 -0.11  0.98 -0.   -0.21 -0.01 -0.   -0.01]

NN Predicts:  Shirt
Actual:  Coat



Prediction:  [0.021221568151759768, 0.6679810078675004, 0.010364486779740384, 0.08219541853201057, 0.007339634926353038, 0.0076873733514327865, 0.050339181694201825, 0.006834717823717614, 0.025601332009476837, 0.019372371615171413]
Actual:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.021221568151759768, 




Prediction:  [0.08157483719099089, 0.22397497353865015, 0.06852252117409549, 0.018186053439338656, 0.057770603998245997, 0.0011394046912974079, 0.2419555642299048, 0.03131140739864653, 0.002882794108085291, 0.0047005009865404125]
Actual:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.08157483719099089, 0.7760250264613499, -0.06852252117409549, -0.018186053439338656, -0.057770603998245997, -0.0011394046912974079, -0.2419555642299048, -0.03131140739864653, -0.002882794108085291, -0.0047005009865404125]
Prediction Inaccuracy(Rounded):  [-0.08  0.78 -0.07 -0.02 -0.06 -0.   -0.24 -0.03 -0.   -0.  ]

NN Predicts:  Shirt
Actual:  Trouser



Prediction:  [0.04305035808928161, 0.13621711039806278, 0.05971557006768238, 0.02160571614661292, 0.1762590839138135, 0.002521586718880791, 0.8872549341843932, 0.01869935150757859, 0.006824593126410387, 0.0047317710768352015]
Actual:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.04305035808928161, -0.13621711039806278, -0.05


NN Predicts:  Shirt
Actual:  Bag



Prediction:  [0.022118325436662152, 0.058209235386670936, 0.09947795073814915, 0.016830067283073367, 0.2016645016561994, 0.002624112843851402, 0.5664293998635362, 0.008960228915696943, 0.011602466514105363, 0.013376287905737427]
Actual:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.022118325436662152, -0.058209235386670936, -0.09947795073814915, -0.016830067283073367, 0.7983354983438006, -0.002624112843851402, -0.5664293998635362, -0.008960228915696943, -0.011602466514105363, -0.013376287905737427]
Prediction Inaccuracy(Rounded):  [-0.02 -0.06 -0.1  -0.02  0.8  -0.   -0.57 -0.01 -0.01 -0.01]

NN Predicts:  Shirt
Actual:  Coat



Prediction:  [0.004946159910419711, 0.031120423683968498, 0.0008409430157770506, 0.008972135404881612, 0.007020363136560262, 0.03567108303937093, 0.0034759268327784025, 0.5518820675362355, 0.006445219252835551, 0.08218307859429395]
Actual:  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

Prediction Inaccuracy:  [-0.0049461599




Prediction:  [0.0026770957662479037, 0.017873804196498547, 0.0009330394882792238, 0.00088077949130466, 0.000922756022133453, 0.20516332320836467, 0.014429031460852974, 0.7670425703728778, 0.009059497868796769, 0.18070326514119198]
Actual:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.0026770957662479037, -0.017873804196498547, -0.0009330394882792238, -0.00088077949130466, -0.000922756022133453, 0.7948366767916353, -0.014429031460852974, -0.7670425703728778, -0.009059497868796769, -0.18070326514119198]
Prediction Inaccuracy(Rounded):  [-0.   -0.02 -0.   -0.   -0.    0.79 -0.01 -0.77 -0.01 -0.18]

NN Predicts:  Sneaker
Actual:  Sandal



Prediction:  [0.014588466052883733, 0.010085152531633022, 0.19135087380097476, 0.03221128694007074, 0.24115993387375145, 0.013991439679726133, 0.12801729761403724, 0.003637959165829092, 0.027524912843530102, 0.013010455938480943]
Actual:  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

Prediction Inaccuracy:  [-0.014588466052883733, -0.0100851525316330